# Automated Sensing of Red vs. White Wine
_... with the cheapest sensor possible_

#### Data Science 420 BB
#### University of Washington Professional & Continuing Education

#### Homework 1: Low-cost Sensor
#### Leo Salemann, 4/11/18

# LOAD DATA

In [1]:
file <- "./RedWhiteWine.csv"
all_data <- read.csv(file, header=TRUE, sep=",", stringsAsFactors = FALSE)

# If the target column, say column class, is numerical, you need to convert it to factors in R for classification problem
all_data$Class <- factor(all_data$Class)
head(all_data)
dim(all_data)


fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,Class
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,1
7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5,1
7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5,1
11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6,1
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,1
7.4,0.66,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5,1


[1] 6497   13

## Redefine Class 1 as Red and 0 as White

## Drop the quality attribute

In [3]:
library("dplyr")


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [4]:
all_data2 = select(all_data, -quality)
head(all_data2)

fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,Class
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,1
7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,1
7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,1
11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,1
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,1
7.4,0.66,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,1


## Reduce to the features our sensor technology can acutually support

In [5]:
data = select(all_data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides, density, Class)
# data = select(all_data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides, density, Class, Color)
head(data)
dim(data)
summary(data)

fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,density,Class
7.4,0.70,0.00,1.9,0.076,0.9978,1
7.8,0.88,0.00,2.6,0.098,0.9968,1
7.8,0.76,0.04,2.3,0.092,0.9970,1
11.2,0.28,0.56,1.9,0.075,0.9980,1
7.4,0.70,0.00,1.9,0.076,0.9978,1
7.4,0.66,0.00,1.8,0.075,0.9978,1


[1] 6497    7

 fixed.acidity    volatile.acidity  citric.acid     residual.sugar  
 Min.   : 3.800   Min.   :0.0800   Min.   :0.0000   Min.   : 0.600  
 1st Qu.: 6.400   1st Qu.:0.2300   1st Qu.:0.2500   1st Qu.: 1.800  
 Median : 7.000   Median :0.2900   Median :0.3100   Median : 3.000  
 Mean   : 7.215   Mean   :0.3397   Mean   :0.3186   Mean   : 5.443  
 3rd Qu.: 7.700   3rd Qu.:0.4000   3rd Qu.:0.3900   3rd Qu.: 8.100  
 Max.   :15.900   Max.   :1.5800   Max.   :1.6600   Max.   :65.800  
   chlorides          density       Class   
 Min.   :0.00900   Min.   :0.9871   0:4898  
 1st Qu.:0.03800   1st Qu.:0.9923   1:1599  
 Median :0.04700   Median :0.9949           
 Mean   :0.05603   Mean   :0.9947           
 3rd Qu.:0.06500   3rd Qu.:0.9970           
 Max.   :0.61100   Max.   :1.0390           

# BUILD A NAIVE BAYES CLASSIFIER

In [6]:
install.packages("naivebayes")
library(naivebayes)
install.packages("pROC")
library(pROC)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Type 'citation("pROC")' for a citation.

Attaching package: ‘pROC’

The following objects are masked from ‘package:stats’:

    cov, smooth, var



# Make a function that quickly tests NBC accuracy

In [23]:
test_nbc_accuracy <- function(df, plot=FALSE) {
 
    # df = select(df, -Color)
    # split into test and training sets
    nrows <- nrow(df)
    smp_size <- floor(0.75 * nrows)

    set.seed(42)
    train_ind <- sample(seq_len(nrows), size = smp_size)
    
    train <- df[train_ind, ]
    test <- df[-train_ind, ]
    
    # create the Naive Bayes Classifer (nbc)
    nbc <- naive_bayes(Class ~ ., data = train)
    
    if (plot) {
        plot(nbc)
    }
    # Make a prediction & test accuracy
    test_pred <- predict(nbc, test,type = "prob")
    
    ### junky debug stuff
    # print(head (test_pred))
    ### junky debug stuff
    accuracy <- sum(test[,"Class"]==test_pred)/length(test_pred)
    print(paste("Accuracy=", round(accuracy*100,2), "%", sep=""))

    # try to do the AUC
    # print(paste("AUC=", auc(test_pred, df$Class)))

}

## Classification Accuracy, All Attributes

In [24]:
test_nbc_accuracy(all_data)
test_nbc_accuracy(all_data2)

[1] "Accuracy=0.77%"
[1] "Accuracy=0.74%"


## Classification Accuracy, Sensor-supported Attributes

In [25]:
test_nbc_accuracy(data)

[1] "Accuracy=0.46%"


# OPTIMISING ATTRIBUTES

## Simplest Case First: Single-Feature Sensor
Presumably, the cheapest sensor we can design and build is one that samples only a single paremeter. Given our small search space (6 features) and reusable function, we can do a quick check in six lines of code

In [26]:
test_nbc_accuracy(select(data, fixed.acidity, Class))
test_nbc_accuracy(select(data, volatile.acidity, Class))
test_nbc_accuracy(select(data, citric.acid, Class))
test_nbc_accuracy(select(data, residual.sugar, Class))
test_nbc_accuracy(select(data, chlorides, Class))
test_nbc_accuracy(select(data, density, Class))

[1] "Accuracy=0.03%"
[1] "Accuracy=0.06%"
[1] "Accuracy=0%"
[1] "Accuracy=0.06%"
[1] "Accuracy=0.31%"
[1] "Accuracy=0%"


Okay, so that didn't rurn out well; but it was a quick test and we sitll learned something.

## Plot the NBC and Use Some Human Intuition

In [27]:
test_nbc_accuracy(data, plot=TRUE)

[1] "Accuracy=0.46%"


Reviewing the plots, ranked by least similar to most similar
- volatile.acidity
- chlorieds
- fixed.acidity
- density
- residual.sugar
- citric.acid

Let's run another battery of tests, trimming the features off from the bottom and working our way up.

In [28]:
test_nbc_accuracy(select(data, -one_of(c('citric.acid'))))

test_nbc_accuracy(select(data, -one_of(c('citric.acid', 
                                         'residual.sugar'))))

test_nbc_accuracy(select(data, -one_of(c('citric.acid', 
                                         'residual.sugar',
                                         'density'))))

test_nbc_accuracy(select(data, -one_of(c('citric.acid', 
                                         'residual.sugar',
                                         'density',
                                         'fixed.acidity'))))


[1] "Accuracy=0.46%"
[1] "Accuracy=0.43%"
[1] "Accuracy=0.37%"
[1] "Accuracy=0.37%"


We're still not able to beat the accuracy from keeping all the features.  Time to pull out the big guns.

## Load Caret form github

In [2]:
install.packages("devtools")
devtools::install_github('topepo/caret/pkg/caret')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
from URL https://api.github.com/repos/topepo/caret/zipball/master
Installing caret
Installing recipes
Installing broom
Installing psych
'/opt/microsoft/ropen/3.4.1/lib64/R/bin/R' --no-site-file --no-environ  \
  --no-save --no-restore --quiet CMD INSTALL  \
  '/tmp/RtmplCziAr/devtools4022c489a9/psych' --library='/home/nbuser/R'  \
  --install-tests 

Installing tidyr
Installing Rcpp
'/opt/microsoft/ropen/3.4.1/lib64/R/bin/R' --no-site-file --no-environ  \
  --no-save --no-restore --quiet CMD INSTALL  \
  '/tmp/RtmplCziAr/devtools40e3e2281/Rcpp' --library='/home/nbuser/R'  \
  --install-tests 

Installing rlang
'/opt/microsoft/ropen/3.4.1/lib64/R/bin/R' --no-site-file --no-environ  \
  --no-save --no-restore --quiet CMD INSTALL  \
  '/tmp/RtmplCziAr/devtools40247d22b/rlang' --library='/home/nbuser/R'  \
  --install-tests 

Installing tibble
Installing pillar
'/opt/microsoft/ropen/3.4.1/lib64/R/bin/R' --no-site-file --no-

## Try a Feature Plot

In [30]:
install.packages("AppliedPredictiveModeling")
library(AppliedPredictiveModeling)
transparentTheme(trans = .4)
library(caret)
featurePlot(x = data[, 1:6], 
            y = data$Class, 
            plot = "pairs",
            ## Add a key at the top
            auto.key = list(columns = 3))

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


ERROR: Error in library(caret): there is no package called ‘caret’


In [ ]:
# MINIMAL NUMBER OF ATTRIBUTES 

# ----------------------------------------------------------

In [44]:
    df = data
    plot = TRUE

    # split into test and training sets
    nrows <- nrow(df)
    smp_size <- floor(0.75 * nrows)

    set.seed(8720885)
    train_ind <- sample(seq_len(nrows), size = smp_size)
    
    train <- df[train_ind, ]
    test <- df[-train_ind, ]
    
    # create the Naive Bayes Classifer (nbc)
    nbc <- naive_bayes(Class ~ ., data = train)
    
    if (plot) {
        plot(nbc)
    }
    # Make a prediction & test accuracy
    test_pred <- predict(nbc, test,type = "prob")
    
    ### junky debug stuff
    # print(head (test_pred))
    ### junky debug stuff
    accuracy <- sum(test[,"Class"]==test_pred)/length(test_pred)
    print(paste("Accuracy=", round(accuracy*100,2), "%", sep=""))

    # try to do the AUC
    # print(paste("AUC=", auc(test_pred, df$Class)))
    # as.numeric(performance(ROCRpred, "auc")@y.values)


[1] "Accuracy=0.43%"


In [ ]:
Maybe try [this instead](http://blog.revolutionanalytics.com/2016/11/calculating-auc.html)

In [ ]:
Maybe try [this instead](https://www.r-bloggers.com/illustrated-guide-to-roc-and-auc/)

In [52]:
colnames(test_pred) <- c("White", "Red")
head(test_pred, 3)
head(test_pred[,'White'], 3)
head(test_pred[,'Red'], 3)


White,Red
6.104496e-05,0.9999390
1.136686e-03,0.9988633
8.027409e-04,0.9991973


[1] 6.104496e-05 1.136686e-03 8.027409e-04

[1] 0.9999390 0.9988633 0.9991973

In [53]:

whitevec <- ifelse(test_pred[,'White'] > test_pred[,'Red'] , 1, 0)
redvec   <- ifelse(test_pred[,'Red'] > test_pred[,'White'] , 1, 0)
head(whitevec, 3)
head(redvec, 3)



predvec <- ifelse(pred=="Lost", 1, 0)
realvec <- ifelse(realResults=="Lost", 1, 0)
pr <- prediction(predvec, realvec)


[1] 0 0 0

[1] 1 1 1

## Run some Tests

## Run some Tests

## Run some Tests

## Assume the cheapest solution is a single sensor.  Try all the single-feature options.

## Run some Tests

Yeah, that didn't work; but it was a quick test, and would have led to a cheaper product if it did.  Was still worth a shot.

## Do some plots

In [ ]:
test_nbc_accuracy(data, plot=TRUE)

In [ ]:
# Try Dropping chlorides, since it's the opposite of residual sugar

In [ ]:
test_data = select(data, -chlorides)
head(test_data)

In [ ]:
test_nbc_accuracy(test_data)

In [ ]:
# Try Dropping Volatile acidity

In [ ]:
test_data = select(test_data, -volatile.acidity)
head(test_data)

In [ ]:
test_nbc_accuracy(test_data)
test_nbc_accuracy(data)
test_nbc_accuracy(all_data)
test_nbc_accuracy(select(all_data, -quality))

In [ ]:


auc(test_data$, test_data$Class)

# ====================< JUNK >=====================

# Create a Naive Bayes Model, Try a Prediction

In [ ]:
nrows <- nrow(data)
smp_size <- floor(0.75 * nrows)

## set the seed to make your partition reproductible
set.seed(8720885)
train_ind <- sample(seq_len(nrows), size = smp_size)

In [ ]:
train <- data[train_ind, ]
test <- data[-train_ind, ]

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)
plot(nbc)

In [ ]:
install.packages("naivebayes")
library(naivebayes)
nbc <- naive_bayes(Class ~ ., data = train)
plot(nbc)

In [ ]:
# Predict the test data
test_pred <- predict(nbc, test,type = "prob")

accuracy <- sum(test[,"Class"]==test_pred)/length(test_pred)
print(paste("Accuracy=", round(accuracy*100,2), "%", sep=""))

## Pull out a handy plotting function from last quarter

In [ ]:
require(ggplot2)
plot.density = function(df, col, xlabel, title) {
    ggplot(df, aes(col)) + 
          geom_bar() + 
          xlab(xlabel) + 
          ggtitle(title) + 
          theme(plot.title = element_text(hjust = 0.5, size=20, face="bold"),
               axis.title.x = element_text(size=15))
}

In [ ]:
options(repr.plot.width=12, repr.plot.height=4)
plot.density(data, data$Class, "1 = Red?, 2 = White?", "Wine Color")

## Try some Fancy Matrix Plotting

In [ ]:
require(GGally)

plot.matrix = function(df, cols){
    options(repr.plot.width=8, repr.plot.height=8)
    ggpairs(data=df,
            columns=cols,
            upper = list(continuous = "density"),
            lower = list(continuous = "density"),
            cardinality_threshold = 130)
}

In [ ]:
plot.matrix(data, c(1:6,7))

In [ ]:
prediction = predict(nbc, test, type= "prob")
head(prediction)